### CurrencyConverter

In [45]:
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests
from langchain_core.tools import InjectedToolArg
from typing import Annotated

### Make a customize tool

In [46]:
# Make a customize tool

# tool-1
@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'
  response = requests.get(url)
  return response.json()

# tool-2
# @tool
# def convert(base_currency_value: int, conversion_rate: float) -> float:
#     """
#     Given currency convertor rate this function calculates the target currency value from a given base currency value
#     """
#     return base_currency_value * conversion_rate

# tool-2
@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given currency convertor rate this function calculates the target currency value from a given base currency value
    """
    return base_currency_value * conversion_rate 
                              

# InjectedToolArg: --> LLM welll not set the values 
# LLM, do not try to fill this argument. I(the developer/runtime) will inject this value after
# running earlier tools.

In [47]:
convert.args

{'base_currency_value': {'title': 'Base Currency Value', 'type': 'integer'},
 'conversion_rate': {'title': 'Conversion Rate', 'type': 'number'}}

In [48]:
get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1749600001,
 'time_last_update_utc': 'Wed, 11 Jun 2025 00:00:01 +0000',
 'time_next_update_unix': 1749686401,
 'time_next_update_utc': 'Thu, 12 Jun 2025 00:00:01 +0000',
 'base_code': 'USD',
 'target_code': 'INR',
 'conversion_rate': 85.608}

In [49]:
convert.invoke({'base_currency_value':10, 'conversion_rate':85.608})

856.08

### Buinding the tool

In [50]:
# Call the model
llm = ChatGroq(
    model_name="llama3-8b-8192",
    temperature=0.5,
    streaming=False
)

In [51]:
### Buind the tool
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

### Tool Calling

In [52]:
messages = [HumanMessage('What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR')]

In [53]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={})]

In [54]:
ai_message = llm_with_tools.invoke(messages)

In [55]:
messages.append(ai_message)

In [56]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_7qe2',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 10,
   'conversion_factor': {'value': {'name': 'conversion_factor',
     'value': 'result_from_get_conversion_factor'}}},
  'id': 'call_zy1n',
  'type': 'tool_call'}]

In [57]:
# ai_messages.tool_calls


# As llm has taken two questions at one time so out logic has failed as it called the tool but 
# also called the agent as well convertor but llm has its last cut of data thet have given a last rate of currency 
# to improve this we need to import from langcahin_core.tools import InjectedToolArg and typing


# [{'name': 'get_conversion_factor',
#   'args': {'base_currency': 'USD', 'target_currency': 'INR'},
#   'id': 'call_6k84',
#   'type': 'tool_call'},
#  {'name': 'convert',
#   'args': {'base_currency_value': 10, 'conversion_rate': 74.5},
#   'id': 'call_696k',
#   'type': 'tool_call'}]


---

### Explanation of `ai_messages.tool_calls` Output

```python
[
  {
    'name': 'get_conversion_factor',
    'args': {'base_currency': 'USD', 'target_currency': 'INR'},
    'id': 'call_6k84',
    'type': 'tool_call'
  },
  {
    'name': 'convert',
    'args': {'base_currency_value': 10, 'conversion_rate': 74.5},
    'id': 'call_696k',
    'type': 'tool_call'
  }
]
```

### Description:

The LLM handled **two sub-tasks in a single step**, issuing **two tool calls**:

1. `get_conversion_factor` – to fetch the exchange rate from USD to INR.
2. `convert` – to compute the final conversion of 10 USD to INR using a hardcoded rate (74.5 in this case).

### Problem:

The logic failed because the LLM tried to:

* Call the tool (`get_conversion_factor`), **and**
* Use outdated prior knowledge to invoke another tool (`convert`) with a hardcoded `conversion_rate`, **without waiting for the actual rate from the first tool**.

### Suggested Fix:

To resolve this issue, you should enforce **tool call sequencing** using argument injection. Specifically, use:

```python
from langchain_core.tools import InjectedToolArg
from typing import Annotated
```

This allows you to **dynamically inject outputs from one tool into another**, ensuring the second tool (`convert`) receives the correct, up-to-date `conversion_rate`.

---

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_8egf',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency': 'USD',
   'base_currency_value': 10,
   'target_currency': 'INR'},
  'id': 'call_k9ep',
  'type': 'tool_call'}]

In [58]:
import json

for tool_call in ai_message.tool_calls:
    # print(tool_call)
    # execute the 1st tool and get the value of conversion rate
    if tool_call ['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # print(tool_message1)
        # fetch this conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        # append this tool message to messages list
        messages.append(tool_message1)

    # execute the 2nd tool and using the conversion rate from tool-1
    if tool_call['name'] == 'convert':
        # fetcha the current arg
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)

In [59]:
messages

[HumanMessage(content='What is the conversion factor between USD and INR, and based on that can you convert 10 USD to INR', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_7qe2', 'function': {'arguments': '{"base_currency":"USD","target_currency":"INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_zy1n', 'function': {'arguments': '{"base_currency_value":10,"conversion_factor":{"value":{"name":"conversion_factor","value":"result_from_get_conversion_factor"}}}', 'name': 'convert'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 159, 'prompt_tokens': 1047, 'total_tokens': 1206, 'completion_time': 0.1325, 'prompt_time': 0.130860807, 'queue_time': 0.24790182200000002, 'total_time': 0.263360807}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--a06b6f2d-6f5c-4628-a54e-7f6103599f18-0', tool_

In [61]:
# Pass this values to LLM
llm_with_tools.invoke(messages)

AIMessage(content='856.08', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 2736, 'total_tokens': 2740, 'completion_time': 0.003333333, 'prompt_time': 0.34086564, 'queue_time': -0.65317425, 'total_time': 0.344198973}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_179b0f92c9', 'finish_reason': 'stop', 'logprobs': None}, id='run--0dc8c9f1-764c-4384-8ac4-0155ce7093cc-0', usage_metadata={'input_tokens': 2736, 'output_tokens': 4, 'total_tokens': 2740})

In [ ]:
# It is not an AI Agent 